---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [8]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
# def plot_graph(G, weight_name=None):
#     '''
#     G: a networkx G
#     weight_name: name of the attribute for plotting edge weights (if G is weighted)
#     '''
#     %matplotlib notebook
#     import matplotlib.pyplot as plt
    
#     plt.figure()
#     pos = nx.spring_layout(G)
#     edges = G.edges()
#     weights = None
    
#     if weight_name:
#         weights = [int(G[u][v][weight_name]) for u,v in edges]
#         labels = nx.get_edge_attributes(G,weight_name)
#         nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
#         nx.draw_networkx(G, pos, edges=edges, width=weights);
#     else:
#         nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [9]:
def answer_one():
    
    G = nx.read_edgelist('Employee_Movie_Choices.txt', delimiter='\t')
    
    return G
answer_one()

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [10]:
def answer_two():
    
    G = answer_one()
    for n in G.nodes():
        if n in employees:
            G.add_node(n, type="employee")
        else:
            G.add_node(n, type="movie")
    return G
answer_two()

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [11]:
def answer_three():
        
    G = answer_two()
    WPG = bipartite.weighted_projected_graph(G, employees)
    return WPG
answer_three()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [12]:
def answer_four():
        
    #Read in relationship graph and convert to datafram
    RG = nx.read_edgelist('Employee_Relationships.txt', data=[('relationship_score', int)])
    RG_df = pd.DataFrame(RG.edges(data=True), columns=['From', 'To', 'relationship_score'])
        
    #Read in previous graph of movies and employees and convert to datafram
    G = answer_three()
    G_df = pd.DataFrame(G.edges(data=True), columns=['From', 'To', 'movies_score'])
    
    #Make a copy of the movie/employee dataframe for experimentation
    G_copy_df = G_df.copy()
    
    #Change the edge direction to make a directed graph in two directions
    G_copy_df.rename(columns={"From":"From_", "To":"From"}, inplace=True)
    G_copy_df.rename(columns={"From_":"To"}, inplace=True)
    
    #Add the original copy and the experimental copy together to have the two directions
    G_final_df = pd.concat([G_df, G_copy_df])
    print (G_final_df)
    
    #Merge the two dataframes on the From and To fields
    final_df = pd.merge(G_final_df, RG_df, on = ['From', 'To'], how='right')
    
    #Set the movie score equal to the weight
    final_df['movies_score'] = final_df['movies_score'].map(lambda x: x['weight'] if type(x)==dict else None)
    
    #Set the relationship score equal to the relationship_score value from the attributes
    final_df['relationship_score'] = final_df['relationship_score'].map(lambda x: x['relationship_score'])
    
    #Fill all NA values with 0 in movie score column
    final_df['movies_score'].fillna(value=0, inplace=True)
    print (final_df)
    
    #Perform correlation and return
    return final_df['movies_score'].corr(final_df['relationship_score'])

answer_four()

       From       To   movies_score
0      Joan     Andy  {'weight': 1}
1      Joan      Lee  {'weight': 3}
2       Lee     Andy  {'weight': 1}
3    Claude     Andy  {'weight': 1}
4    Claude  Georgia  {'weight': 3}
5      Andy    Frida  {'weight': 1}
6      Andy    Pablo  {'weight': 1}
7      Andy  Georgia  {'weight': 1}
8     Frida    Pablo  {'weight': 2}
9     Frida  Vincent  {'weight': 2}
10    Pablo  Vincent  {'weight': 1}
0      Andy     Joan  {'weight': 1}
1       Lee     Joan  {'weight': 3}
2      Andy      Lee  {'weight': 1}
3      Andy   Claude  {'weight': 1}
4   Georgia   Claude  {'weight': 3}
5     Frida     Andy  {'weight': 1}
6     Pablo     Andy  {'weight': 1}
7   Georgia     Andy  {'weight': 1}
8     Pablo    Frida  {'weight': 2}
9   Vincent    Frida  {'weight': 2}
10  Vincent    Pablo  {'weight': 1}
       From       To  movies_score  relationship_score
0      Joan      Lee           3.0                  70
1    Claude  Georgia           3.0                  90
2      

0.7883962221733474